In [145]:
import pandas as pd
import numpy as np 
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', None)

# Data loading

In [146]:
data_path = Path().absolute().parent/'data'
geo_info = pd.read_csv(data_path/'geo_info.csv', sep=';')
referer_vectors = pd.read_csv(data_path/'referer_vectors.csv', sep=';')
test = pd.read_csv(data_path/'test.csv', sep=';')
test_users = pd.read_csv(data_path/'test_users.csv', sep=';')
train = pd.read_csv(data_path/'train.csv', sep=';')
train_labels = pd.read_csv(data_path/'train_labels.csv', sep=';')

# Initial analysis

In [147]:
train.info()
display(train.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   request_ts  750000 non-null  int64 
 1   user_id     750000 non-null  object
 2   referer     750000 non-null  object
 3   geo_id      750000 non-null  int64 
 4   user_agent  749999 non-null  object
dtypes: int64(2), object(3)
memory usage: 28.6+ MB


,request_ts,user_id,referer,geo_id,user_agent
522332,1700951708,7bcbc1af18eead54a1299667249fcbdf,https://635e50c/16d7c751,3663,"{'browser': 'Chrome Mobile', 'browser_version'..."
654631,1701004026,405339423995c41bd64e36d345401dca,https://9ac1e4f/172507b2,4984,"{'browser': 'Yandex Browser', 'browser_version..."
319758,1701014478,11575e55f1d4011184df9e57419da079,https://7b8d3ae/1434f049,7376,"{'browser': 'Chrome Mobile', 'browser_version'..."
487520,1701022565,9e0fe2f4f0d8ddd3f800cb999446af6c,https://69df421/1200fda5,3663,"{'browser': 'Chrome Mobile', 'browser_version'..."
217998,1701014595,f956e5d9f25479e98521129b2b402b62,https://a9b8175/167139aa,3871,"{'browser': 'Chrome Mobile iOS', 'browser_vers..."


In [148]:
geo_info.info()
display(geo_info.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   geo_id       5533 non-null   int64 
 1   country_id   5533 non-null   object
 2   region_id    3826 non-null   object
 3   timezone_id  5533 non-null   object
dtypes: int64(1), object(3)
memory usage: 173.0+ KB


,geo_id,country_id,region_id,timezone_id
4427,9079,f9129d,NaN,d20730
1618,6600,a175c5,NaN,df925b
2412,741,c31b4e,607a45,e56e80
4801,2523,c31b4e,f66ff,f6155e
2173,6105,c31b4e,23f9c2,f6155e


In [149]:
referer_vectors.info()
display(referer_vectors.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   component0  200000 non-null  int64 
 1   component1  200000 non-null  int64 
 2   component2  200000 non-null  int64 
 3   component3  200000 non-null  int64 
 4   component4  200000 non-null  int64 
 5   component5  200000 non-null  int64 
 6   component6  200000 non-null  int64 
 7   component7  200000 non-null  int64 
 8   component8  200000 non-null  int64 
 9   component9  200000 non-null  int64 
 10  referer     200000 non-null  object
dtypes: int64(10), object(1)
memory usage: 16.8+ MB


,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,referer
185888,19013,-2594,5403,6770,2311,11501,17483,8693,-4506,7899,https://7b8d3ae/1702147c
108919,19240,-2525,14965,5352,7808,5401,8562,15289,-5621,-1355,https://73e223f/14cabe24
72031,-348,8595,11350,12441,10749,2903,12771,13282,768,13422,https://72879b4/130de7f1
78324,12960,15118,4196,-12657,8341,4740,-5824,12993,-8184,13794,https://b5380d6/14c25657
188847,14097,-1219,17279,8131,13912,9729,10559,2228,6543,957,https://bbc64b8/14356938


In [150]:
train_labels.info()
display(train_labels.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  500000 non-null  object
 1   target   500000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.6+ MB


,user_id,target
482840,3a531ffc7b782124cae2a7c78efc7308,0
107942,5caddfae1baa0e003955ccc6354ead48,1
155649,c80bb716b756650e61b2d7aa55707b66,1
210709,9ed9f7f8f35a4071bcc1ee3b1fec8b22,1
4822,446640d826279b563d91cae7650d8874,0


# Data Processing

In [151]:
# Объединим таблицы
train = pd.merge(train, geo_info, on='geo_id')
train = pd.merge(train, referer_vectors, on='referer')
train = pd.merge(train, train_labels, on='user_id')
train = train.drop(['geo_id', 'referer'], axis=1)

In [152]:
# Найдем и удалим дубликаты
print('Количество дубликатов:', train.duplicated().sum())
train = train.drop_duplicates()

Количество дубликатов: 9454


In [153]:
# Найдем и удалим пропуски
print('Кол-во пропусков:\n', train.isnull().sum())
train = train.dropna(axis=0, how='any')

Кол-во пропусков:
 request_ts         0
user_id            0
user_agent         1
country_id         0
region_id      49903
timezone_id        0
component0         0
component1         0
component2         0
component3         0
component4         0
component5         0
component6         0
component7         0
component8         0
component9         0
target             0
dtype: int64


In [154]:
train[(train['country_id']=='c31b4e') & (train['timezone_id']=='e56e80')].head(3)

,request_ts,user_id,user_agent,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,target
1,1700986581,46a5f128fd569c764a92c2eaa788095e,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,44520b,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
3,1700992803,af735816ca19115431ae3d89518c8c91,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
5,1700956244,b74aaac38e44f90517d3e217fa118cec,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,1fbfa5,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0


In [155]:
train[(train['region_id']=='3c9dca')].head(3)

,request_ts,user_id,user_agent,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,target
3,1700992803,af735816ca19115431ae3d89518c8c91,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
804,1701023748,5b3cdd0eba30838fb01deb702845cdf4,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,13294,3884,10452,-1916,11369,6138,-2472,10123,3029,21118,1
1024,1701019068,18232c8806188f0adeb265e4600223af,"{'browser': 'Chrome', 'browser_version': '118....",c31b4e,3c9dca,e56e80,16220,2266,9404,-3442,11485,4129,-3919,6506,2556,21868,0


In [156]:
train = train.drop(['country_id', 'timezone_id'], axis=1)

Признаки country_id и timezone_id не информативны, поскольку при их фиксированных значениях параметр region_id имеет различные значения. А фиксированному region_id соответствуют одни и те же единственные показатели country_id и timezone_id. Поэтому мы можем исключить эти столбцы.

In [157]:
# Раскроем словарь столбца user_agent
import ast
list = ['browser', 'browser_version', 'os', 'os_version']
for key in list:
    train[key] = train['user_agent'].apply(lambda x: ast.literal_eval(x)[key])
train = train.drop('user_agent', axis=1)

In [158]:
# Ввиду высокой смысловой корреляции между browser и os,удаляем последний, тк он, к тому же, менее информативен
train = train.drop(['os', 'browser_version', 'os_version'], axis=1)

In [159]:
train = train.drop(['user_id'], axis=1)

In [160]:
# выделим топ 25 значений столбца browser
top_25 = [x for x in train['browser'].value_counts().sort_values(ascending=False).head(25).index]
train['browser'] = train['browser'].apply(lambda x: x if x in top_25 else 'Other')

In [161]:
# преобразуем столбец дата-время request_ts
train['request_date'] = pd.to_datetime(train['request_ts'], unit='s')
train.insert(loc=1,column='request_year', value=train['request_date'].dt.year)
train.insert(loc=2, column='request_month', value=train['request_date'].dt.month)
train.insert(loc=3, column='request_dayofweek', value=(train['request_date'].dt.dayofweek)+1)
train.insert(loc=4, column='request_hour', value=train['request_date'].dt.hour)

print('Number of unique year: ', train['request_year'].nunique())
print('Number of unique month: ', train['request_month'].nunique())
print('Number of unique dayofweek: ', train['request_dayofweek'].nunique())
print('\n',train['request_dayofweek'].value_counts())
print('\nNumber of unique hour: ', train['request_hour'].nunique())

# Удалим стоблцы year, month, тк они не информативны и содержат единственное значение. Столбец request_dayofweek, поскольку он содержит только дни 6 и 7, которые коррелируют как выходные. А также исходный request_ts и request_date
train = train.drop(['request_year', 'request_month', 'request_ts', 'request_dayofweek', 'request_date'], axis=1)

Number of unique year:  1
Number of unique month:  1
Number of unique dayofweek:  2

 request_dayofweek
7    510348
6     31584
Name: count, dtype: int64

Number of unique hour:  24


In [162]:
# нормализуем столбцы
numeric_features = ['component0', 'component1', 'component2', 'component3', 'component4', 'component5', \
                    'component6', 'component7', 'component8', 'component9']
time_feature = ['request_hour']
mean_encoding_feature = ['region_id']


def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))
def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

one_hot_encoder = OneHotEncoder(drop='first', handle_unknown='ignore',sparse_output=False)

target = TargetEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', one_hot_encoder, ['browser']),
        ("hour_sin", sin_transformer(24), ["request_hour"]),
        ("hour_cos", cos_transformer(24), ["request_hour"]),
        ('trgt', target, ['region_id'])
    ])

preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['component0', 'component1', 'component2',
                                  'component3', 'component4', 'component5',
                                  'component6', 'component7', 'component8',
                                  'component9']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['browser']),
                                ('hour_sin',
                                 FunctionTransformer(func=<function sin_transformer.<locals>.<lambda> at 0x0000026F96DD3740>),
                                 ['request_hour']),
                                ('hour_cos',
                                 FunctionTransformer(func=<function cos_transformer.<locals>.<lambda> at 0x0000026F96DD3600>),
                                 ['request_hour']),
                                ('trgt', TargetEncoder(), ['region_id'])])

In [163]:
# определим целевую переменную и обучающий набор
X = train.drop('target', axis=1)
y = train['target']

In [164]:
df_pandas = preprocessor.fit_transform(X, y)

In [165]:
# полученное преобразование
display(df_pandas)

,num__component0,num__component1,num__component2,num__component3,num__component4,num__component5,num__component6,num__component7,num__component8,num__component9,cat__browser_Chrome Mobile,cat__browser_Chrome Mobile WebView,cat__browser_Chrome Mobile iOS,cat__browser_Edge,cat__browser_Edge Mobile,cat__browser_Facebook,cat__browser_Firefox,cat__browser_Firefox Mobile,cat__browser_Google,cat__browser_Instagram,cat__browser_Iron,cat__browser_Mail.ru Chromium Browser,cat__browser_MiuiBrowser,cat__browser_Mobile Safari,cat__browser_Mobile Safari UI/WKWebView,cat__browser_Opera,cat__browser_Opera Mobile,cat__browser_Other,cat__browser_Pinterest,cat__browser_Safari,cat__browser_Samsung Internet,cat__browser_UC Browser,cat__browser_Yandex Browser,cat__browser_YandexModule2,cat__browser_YandexSearch,hour_sin__request_hour,hour_cos__request_hour,trgt__region_id
0,0.140963,-0.366928,2.096905,-0.637888,-2.435214,0.388581,-0.887897,-1.064147,-0.495173,0.897532,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.707107,-7.071068e-01,0.437392
1,0.140963,-0.366928,2.096905,-0.637888,-2.435214,0.388581,-0.887897,-1.064147,-0.495173,0.897532,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866025,-5.000000e-01,0.462935
2,0.256275,-0.582120,0.560760,-1.362967,0.385429,-0.682069,-0.719194,0.799518,-0.058250,1.330400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.707107,-7.071068e-01,0.444051
3,0.140963,-0.366928,2.096905,-0.637888,-2.435214,0.388581,-0.887897,-1.064147,-0.495173,0.897532,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,-8.660254e-01,0.449262
4,0.140963,-0.366928,2.096905,-0.637888,-2.435214,0.388581,-0.887897,-1.064147,-0.495173,0.897532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.000000,-1.836970e-16,0.463491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601284,0.626566,0.867392,-0.255232,-0.219809,0.678858,1.186067,-0.289825,-2.095539,0.326193,0.251014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.258819,9.659258e-01,0.445258
601285,0.231318,1.446953,0.865177,-0.609979,-0.499377,0.760707,0.119682,1.404362,-0.256234,-0.627144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,-8.660254e-01,0.457205
601286,0.400753,-1.778774,0.929156,1.367523,-0.451179,-0.550861,1.271458,0.756781,-0.570394,-1.797850,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866025,-5.000000e-01,0.471546
601287,0.400753,-1.778774,0.929156,1.367523,-0.451179,-0.550861,1.271458,0.756781,-0.570394,-1.797850,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,6.123234e-17,0.473448
